In [1]:
import pandas as pd
import numpy as np
from scipy.special import expit # Sigmoid function
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from tqdm import tqdm

file_injuryHx = '../../data/IMPACT/InjuryHX_Motor_Pupils_Hypotension_Hypoxia.csv'
file_imaging  = '../../data/IMPACT/Imaging_Marshall_Hematoma_Hemorrhage.csv'

rootPath = '../../'
dataPath = rootPath + 'data_preparation/biomarkers_acute_sample_462/'
dataFile462 = dataPath + 'master_acute_sample_462_X6.xlsx'
dataFile373 = dataPath + 'master_acute_sample_462_X7_helsinki.xlsx'

GOSEFile = rootPath + 'data/_neurobot_outcome_GOSE_scores.csv'
QoLIBRIOSFile = rootPath + 'data/_neurobot_outcome_QoLIBRIOS.csv'
deathFile = rootPath + 'data/Subject Death Information.xlsx'
neuroworseningFile = rootPath + 'data/_neurobot_neuroworsening_secondinsults.csv'
GOAT_RAVLT_File = rootPath + 'data/_neurobot_outcome_GOAT_RAVLT_scores.csv'
RPQFile = rootPath + 'data/_neurobot_OutcomesRPQ.csv'

resultFolder462 = 'results_462_4-25/'
resultFolder373 = 'results_462_5-5_helsinki/'
resultFolder373d1 = 'result_d1_CTpos/'

rank462 = 'acute_sample462_X6_rank_1.xlsx'
rank373 = 'acute_sample462_X7_rank_1.xlsx'
rank373d1 = 'acute_sample373_CTpos_rank_1.xlsx'

biomarkers = ['GFAP', 'NFL', 'Tau', 'UCH-L1', 'S100B', 'NSE']
biomarker5 = ['GFAP', 'NFL', 'Tau', 'UCH-L1', 'S100B']

gOSEdict = {'GOSE3month': 3, 'GOSE6month': 6, 'GOSE12month': 12}
goodGOSE = {
    1:0, 2:0, 3:0, 4:0,
    5:0, 6:0, 7:1, 8:1
}
poorGOSE = {
    1:1, 2:1, 3:1, 4:1,
    5:0, 6:0, 7:0, 8:0
}

algs = {'km':'kmeans', 'ga':'gaussian_mixture', 'sp':'spectral', 'sk' : 'skm', 'ag' : 'agglomerative'}

ftsize = 24

controldataf = rootPath + 'data/Center-6-markers -Controls.xlsx'

In [2]:
# IMPACT raw data pre-processing
dfinj = pd.read_csv(file_injuryHx, header=0)
dfimg = pd.read_csv(file_imaging, header=0)

#check duplicated GUPI
has_duplicates = dfinj['InjuryHx.GUPI'].duplicated().any()
print(f"dfinj duplicated: {has_duplicates} shape {dfinj.shape}")
dfimg = dfimg.drop_duplicates(keep='first')
dfimg = dfimg[dfimg['Imaging.Timepoint'] == 'CT Early'] # Select the early CT
has_duplicates = dfimg['Imaging.GUPI'].duplicated().any()
print(f"dfimg duplicated: {has_duplicates} shape {dfimg.shape}")
# dfimg['Imaging.GUPI'].value_counts()
dfinj = dfinj.set_index('InjuryHx.GUPI', drop=True)
dfinj.index.name = 'subjectId'
dfimg = dfimg.set_index('Imaging.GUPI', drop=True)
dfimg.index.name = 'subjectId'

dfinj duplicated: False shape (4509, 5)
dfimg duplicated: False shape (4221, 5)


In [3]:
#Control data
df1 = pd.read_excel(controldataf, sheet_name='Sheet1', usecols='A, B, C', header=0, index_col=0)
df1.columns = ['NSE', 'S100B']
df1 = df1.dropna()
df2 = pd.read_excel(controldataf, sheet_name='Sheet1', usecols='E, F, G, H, I', header=0, index_col=0)
df2.columns = ['GFAP', 'NFL', 'Tau', 'UCH-L1']
df2['GFAP'] = df2['GFAP'] / 1000 # Unit conversion form pg/ml -> ng/ml
ctldf = pd.merge(df1, df2, left_index=True, right_index=True, how='outer').reset_index(drop=True).dropna(how='all')
# print(ctldf)

In [4]:
class result:
    rootPath = '../../'
    def __init__(self, resultfolder, dataFile, rankFile):
        if not resultfolder.endswith('/'):
            folder += '/'
        self.resultPath = rootPath + resultfolder
        self.clusterPath = self.resultPath + 'cluster/'
        self.rankingPath = self.resultPath + 'ranking/'
        self.statsPath = self.resultPath + 'stats/'
        self.rankFile = self.rankingPath + rankFile
        self.dataFile = dataFile
        self.df = {}
        for bio in biomarkers:
            dfin = pd.read_excel(self.dataFile, sheet_name=bio+' KNN imputed normalized', header=0, index_col=0)
            # dfin = pd.read_excel(self.dataFile, sheet_name=bio, header=0, index_col=0)
            ctldf.index = dfin.index[:len(ctldf)]
            self.df[bio] = dfin.iloc[:, -5:]
            self.df[bio]['ctrl'] = ctldf[bio]
            self.dfindex = dfin.index
        return
    
    def getoutcome(self, biomarker, rank=1):
        outcome = pd.DataFrame()
        dfin = pd.read_excel(self.dataFile, sheet_name=biomarker, header=0, index_col=0)
        dfRank = pd.read_excel(self.rankFile, sheet_name=biomarker, header=0)
        fst_cluster = dfRank.iloc[rank - 1,0]
        fst_a, fst_c = fst_cluster.split('_')
        labelFile = self.clusterPath + biomarker + '/' + biomarker + '_' + algs[fst_a] + '_' + fst_c + '.csv'
        label = pd.read_csv(labelFile, header=0)
        label.index = self.dfindex
        df1 = dfin.iloc[:, -4]
        grouped = df1.groupby(label['Cluster']).mean()
        if grouped[1].mean() > grouped[2].mean():
            label['Cluster'] = label['Cluster'].replace({1:2, 2:1})
        outcome['Cluster'] = label
        clusterTitle = biomarker + ' ' + algs[fst_a] + ' ' + fst_c
        
        dfGOSE = pd.read_csv(GOSEFile, header=0 )
        dfGOSE = dfGOSE.iloc[:, [0, -3, -2, -1]].drop_duplicates()
        dfGOSE.set_index('subjectId', inplace=True)
        dfGOSE.replace('2_or_3', 3, inplace=True)
        dfGOSE = dfGOSE.apply(pd.to_numeric, errors='coerce')
        outcome['GOSE3month'] = dfGOSE['Subject.GOSE3monthEndpointDerived']
        outcome['GOSE6month'] = dfGOSE['Subject.GOSE6monthEndpointDerived']
        outcome['GOSE12month'] = dfGOSE['Subject.GOSE12monthEndpointDerived']
        
        df = pd.read_csv(QoLIBRIOSFile, header=0, index_col=0)
        df_qolib = df.pivot(columns='Outcomes.Timepoint', values='Outcomes.QoLIBRIOSTotalScore')        
        outcome['QoLIB3month'] = df_qolib['3mo']
        outcome['QoLIB6month'] = df_qolib['6mo']
        outcome['QoLIB12month']= df_qolib['12mo']
        # outcome['QoLIB24month']= df_qolib['24mo']
        # outcome['QoLIB2week']= df_qolib['2wk']

        df = pd.read_excel(deathFile, sheet_name='Subject Death Information', header=0, index_col=0)
        df['DateInj'] = pd.to_datetime(df['DateInj'])
        df['DeathDate'] = pd.to_datetime(df['DeathDate'])
        df['Deathbydays'] = (df['DeathDate'] - df['DateInj']).dt.days
        outcome['Deathbydays'] = df['Deathbydays']
        outcome['DeathCause'] = df['DeathCause'].replace({4:3}) # no #3 cause, replace all 4 with 3
        
        dfin = pd.read_excel(self.dataFile, sheet_name=biomarkers[0], header=0, index_col=0)
        outcome['GCS'] = dfin['GCS']
        
        dfwsn = pd.read_csv(neuroworseningFile, header=0)
        for k, v in {'InjuryHx.EDSecondInsultsNeuroWorse':'NeuroWorse'}.items():
        # for k, v in {'InjuryHx.EDComplEventHypoxia':'NeuroWorse'}.items():
            dfw = dfwsn[['subjectId', k]].drop_duplicates().set_index('subjectId')
            dfw[k] = dfw[k].replace({88: np.NaN})
            outcome[v] = dfw[k]
        outcome['Age'] = dfin['Age']
        return clusterTitle, outcome


In [5]:
# IMPACT look-up tables
Core_mortal = {
    'Const' : -3.109,
    'Age' : 0.034,
    'Motor' : {
        1 : 1.447,      # No motor response
        2 : 1.397,      # Extension to pain
        3 : 0.797,      # Flexion to pain
        4 : 0.390,      # Withdrawal from pain
        5 : 0,          # Localizes pain
        6 : 0,          # Obeys commands
        -1 : 0.522,     # Missing
        },
    'Pupils' : {
        0 : 0,          # Both reactive
        1 : 0.514,      # One reactive
        2 : 1.239       # Neither reactive
        },    
}
Core_unfavor = {
    'Const' : -2.644,
    'Age' : 0.038,
    'Motor' : {
        1 : 1.393,      # No motor response
        2 : 2.087,      # Extension to pain
        3 : 1.266,      # Flexion to pain
        4 : 0.632,      # Withdrawal from pain
        5 : 0,          # Localizes pain
        6 : 0,          # Obeys commands
        -1 : 0.885,     # Missing
        },
    'Pupils' : {
        0 : 0,          # Both reactive
        1 : 0.592,      # One reactive
        2 : 1.216       # Neither reactive
        },    
}
Ext_mortal = {
    'Const' : -3.787,
    'Age' : 0.032,
    'Motor' : {
        1 : 1.205,      # No motor response
        2 : 1.207,      # Extension to pain
        3 : 0.746,      # Flexion to pain
        4 : 0.313,      # Withdrawal from pain
        5 : 0,          # Localizes pain
        6 : 0,          # Obeys commands
        -1 : 0.425,     # Missing
        },
    'Pupils' : {
        0 : 0,          # Both reactive
        1 : 0.334,      # One reactive
        2 : 0.970       # Neither reactive
        },
    'MarshallCT' : {
        1 : -0.298,     # Diffuse injury I
        2 : 0,          # Diffuse injury II
        3 : 0.774,      # Diffuse injury III
        4 : 0.774,      # Diffuse injury IV
        5 : 0.651,      # evacuated mass lesion
        6 : 0.651,      # non-evacuated mass lesion
        },
    'Hemorrhage' : {            # Traumatic subarachnoid hemorrhage on CT
        'present' : 0.606,
        'absent'  : 0,
        'uninterpretable' : 0,
        },
    'Hematoma' : {              # Epidural hematoma on CT
        'present' : -0.379,
        'absent'  : 0,
        'uninterpretable' : 0,
        },
    'Hypoxia' : {
        0 : 0,          # No
        88: 0,          # Unknown
        1 : 0.237,      # Yes
        2 : 0.237,      # Suspect
        },
    'Hypotension' : {
        0 : 0,          # No
        88: 0,          # Unknown
        1 : 0.667,      # Yes
        2 : 0.667,      # Suspect
        }
}
Ext_unfavor = {
    'Const' : -3.023,
    'Age' : 0.033,
    'Motor' : {
        1 : 1.218,      # No motor response
        2 : 1.852,      # Extension to pain
        3 : 1.185,      # Flexion to pain
        4 : 0.575,      # Withdrawal from pain
        5 : 0,          # Localizes pain
        6 : 0,          # Obeys commands
        -1 : 0.837,     # Missing
        },
    'Pupils' : {
        0 : 0,          # Both reactive
        1 : 0.442,      # One reactive
        2 : 1.003       # Neither reactive
        },
    'MarshallCT' : {
        1 : -0.530,     # Diffuse injury I
        2 : 0,          # Diffuse injury II
        3 : 0.543,      # Diffuse injury III
        4 : 0.543,      # Diffuse injury IV
        5 : 0.497,      # evacuated mass lesion
        6 : 0.497,      # non-evacuated mass lesion
        },
    'Hemorrhage' : {            # Traumatic subarachnoid hemorrhage on CT
        'present' : 0.567,
        'absent'  : 0,
        'uninterpretable' : 0,
        },
    'Hematoma' : {              # Epidural hematoma on CT
        'present' : -0.572,
        'absent'  : 0,
        'uninterpretable' : 0,
        },
    'Hypoxia' : {
        0 : 0,          # No
        88: 0,          # Unknown
        1 : 0.316,      # Yes
        2 : 0.316,      # Suspect
        },
    'Hypotension' : {
        0 : 0,          # No
        88: 0,          # Unknown
        1 : 0.614,      # Yes
        2 : 0.614,      # Suspect
        }
}

In [6]:
res = result(resultfolder=resultFolder373, dataFile=dataFile373, rankFile=rank373)
# res = result(resultfolder=resultFolder373d1, dataFile=dataFile373, rankFile=rank373d1)

ctitle, outcome = res.getoutcome(biomarker=biomarkers[0])
df = outcome.copy()
for bio in biomarkers:
    ctitle, outcome = res.getoutcome(biomarker=bio)
    df[bio] = outcome['Cluster'] - 1
    for i in range(1, 6):
        df[f"{bio}_{i}"] = res.df[bio][f"{bio}_{i}"]

df[['Motor', 'Pupils']] = dfinj[['InjuryHx.GCSMotorBaselineDerived', 'InjuryHx.PupilsBaselineDerived']]
df[['MarshallCT', 'Hemorrhage', 'Hematoma']] = dfimg[['Imaging.MarshallCTClassification', 'Imaging.TraumaticSubarachnoidHemorrhage', 'Imaging.EpiduralHematoma']]
df[['Hypoxia', 'Hypotension']] = dfinj[['InjuryHx.EDComplEventHypoxia', 'InjuryHx.EDComplEventHypotension']]
# df.to_csv('../../IMPACT/Outcome.csv', header=True, index=True)
df['core_mortal']  = expit(Core_mortal['Const'] + df['Age']*Core_mortal['Age'] + df['Motor'].map(Core_mortal['Motor']).fillna(Core_mortal['Motor'][-1]) + df['Pupils'].map(Core_mortal['Pupils']))
df['core_unfavor'] = expit(Core_unfavor['Const'] + df['Age']*Core_unfavor['Age'] + df['Motor'].map(Core_unfavor['Motor']).fillna(Core_unfavor['Motor'][-1]) + df['Pupils'].map(Core_unfavor['Pupils']))
df['ext_mortal']  = expit(Ext_mortal['Const'] + df['Age']*Ext_mortal['Age'] + df['Motor'].map(Ext_mortal['Motor']).fillna(Ext_mortal['Motor'][-1]) + df['Pupils'].map(Ext_mortal['Pupils'])
                    + df['MarshallCT'].map(Ext_mortal['MarshallCT']) + df['Hemorrhage'].map(Ext_mortal['Hemorrhage']) + df['Hematoma'].map(Ext_mortal['Hematoma'])
                    + df['Hypotension'].map(Ext_mortal['Hypotension']) + df['Hypoxia'].map(Ext_mortal['Hypoxia']) )
df['ext_unfavor'] = expit(Ext_unfavor['Const'] + df['Age']*Ext_unfavor['Age'] + df['Motor'].map(Ext_unfavor['Motor']).fillna(Ext_unfavor['Motor'][-1]) + df['Pupils'].map(Ext_unfavor['Pupils'])
                    + df['MarshallCT'].map(Ext_unfavor['MarshallCT']) + df['Hemorrhage'].map(Ext_unfavor['Hemorrhage']) + df['Hematoma'].map(Ext_unfavor['Hematoma'])
                    + df['Hypotension'].map(Ext_unfavor['Hypotension']) + df['Hypoxia'].map(Ext_unfavor['Hypoxia']) )   

# print(f"GOSE 6 month NaN {df['GOSE6month'].isna().sum()}")
df = df.dropna(subset=['GOSE6month', 'core_mortal', 'ext_mortal'])
# for col in ['core_mortal', 'core_unfavor', 'ext_mortal', 'ext_unfavor']:
#     print(f"{col} NaN {df[col].isna().sum()}")
print(df.columns, df.shape)
df['target_mortal'] = df['GOSE6month'].map(lambda x: 1 if x == 1 else 0)
df['target_unfavor'] = df['GOSE6month'].map(lambda x: 1 if x < 5 else 0)

cluster_sum = df[biomarker5].sum(axis=1)
df['composite'] = 2
df.loc[cluster_sum == 0, 'composite'] = 0
df.loc[cluster_sum == 5, 'composite'] = 1

df1 = df[df['composite'] != 2].copy()


# results = []

# bio = 'composite'
# y = df1['target_unfavor']
# X1 = sm.add_constant(df1['ext_unfavor'])
# model1 = sm.Logit(y, X1).fit()    
# X2 = sm.add_constant(df1[bio])
# model2 = sm.Logit(y, X2).fit()    
# X3 = sm.add_constant(df1[['ext_unfavor', bio]])
# model3 = sm.Logit(y, X3).fit()    
# print(bio)
# y_pred_prob = model1.predict()
# auc1 = roc_auc_score(y, y_pred_prob)
# print(f"IMPACT AUC: {auc1:.2f}")
# y_pred_prob = model2.predict()
# auc2 = roc_auc_score(y, y_pred_prob)
# print(f"Composite AUC: {auc2:.2f}")
# y_pred_prob = model3.predict()
# auc3 = roc_auc_score(y, y_pred_prob)
# print(f"COmbined AUC: {auc3:.2f}")

# results.append({'biomarker': bio, 'IMPACT AUC': f"{auc1:.2f}", 'Cluster AUC': f"{auc2:.2f}", 'IMPACT + Cluster': f"{auc3:.2f}"})



# for bio in biomarkers:
#     y = df['target_unfavor']
#     X1 = sm.add_constant(df['ext_unfavor'])
#     model1 = sm.Logit(y, X1).fit()    
#     X2 = sm.add_constant(df[bio])
#     model2 = sm.Logit(y, X2).fit()    
#     X3 = sm.add_constant(df[['ext_unfavor', bio]])
#     model3 = sm.Logit(y, X3).fit()    
#     print(bio)
#     y_pred_prob = model1.predict()
#     auc1 = roc_auc_score(y, y_pred_prob)
#     # print(f"IMPACT AUC: {auc:.2f}")
#     y_pred_prob = model2.predict()
#     auc2 = roc_auc_score(y, y_pred_prob)
#     # print(f"{bio} AUC: {auc:.2f}")
#     y_pred_prob = model3.predict()
#     auc3 = roc_auc_score(y, y_pred_prob)
#     # print(f"Combined AUC: {auc:.2f}")
#     results.append({'biomarker': bio, 'IMPACT AUC': f"{auc1:.2f}", 'Cluster AUC': f"{auc2:.2f}", 'IMPACT + Cluster': f"{auc3:.2f}"})

#     # outcome[['core_mortal', 'core_unfavor', 'ext_mortal', 'ext_unfavor']] = df[['core_mortal', 'core_unfavor', 'ext_mortal', 'ext_unfavor']]
#     # break
# # outcome
# df_res = pd.DataFrame(results)
# df_res

Index(['Cluster', 'GOSE3month', 'GOSE6month', 'GOSE12month', 'QoLIB3month',
       'QoLIB6month', 'QoLIB12month', 'Deathbydays', 'DeathCause', 'GCS',
       'NeuroWorse', 'Age', 'GFAP', 'GFAP_1', 'GFAP_2', 'GFAP_3', 'GFAP_4',
       'GFAP_5', 'NFL', 'NFL_1', 'NFL_2', 'NFL_3', 'NFL_4', 'NFL_5', 'Tau',
       'Tau_1', 'Tau_2', 'Tau_3', 'Tau_4', 'Tau_5', 'UCH-L1', 'UCH-L1_1',
       'UCH-L1_2', 'UCH-L1_3', 'UCH-L1_4', 'UCH-L1_5', 'S100B', 'S100B_1',
       'S100B_2', 'S100B_3', 'S100B_4', 'S100B_5', 'NSE', 'NSE_1', 'NSE_2',
       'NSE_3', 'NSE_4', 'NSE_5', 'Motor', 'Pupils', 'MarshallCT',
       'Hemorrhage', 'Hematoma', 'Hypoxia', 'Hypotension', 'core_mortal',
       'core_unfavor', 'ext_mortal', 'ext_unfavor'],
      dtype='object') (305, 59)


In [ ]:
bio = 'composite'
y_true = {}
y_pred = {}
for bio in biomarkers:
    df[f"{bio}_mortal"]  = df[f"{bio}"].map({0:0.08, 1: 0.33})
    df[f"{bio}_unfavor"] = df[f"{bio}"].map({0:0.30, 1: 0.70})
AIC_LRT_table = []
for IMPACT_model in ['core', 'ext']:
    for IMPACT_out in ['unfavor', 'mortal']:
        bio = 'composite'
        y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"] = df1[f"target_{IMPACT_out}"]
        X1 = sm.add_constant(df1[f"{IMPACT_model}_{IMPACT_out}"])
        model1 = sm.Logit(y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"], X1).fit()
        X2 = sm.add_constant(df1[[f"{IMPACT_model}_{IMPACT_out}", bio]])
        model2 = sm.Logit(y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"], X2).fit()
        X3 = sm.add_constant(df1[bio])
        model3 = sm.Logit(y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"], X3).fit()
        
        y_pred[f"{IMPACT_model}_{IMPACT_out}_{bio}_IMPACT"] = model1.predict()
        y_pred[f"{IMPACT_model}_{IMPACT_out}_{bio}_COMBINE"] = model2.predict()
        
        ll_IMPACT  = model1.llf
        ll_COMBINE = model2.llf
        ll_Cluster = model3.llf
        
        ll_null = model1.llnull
        nobs = model1.nobs

        R2_nagelkerke_IMPACT  = (1 - np.exp((2 / nobs) * (ll_null - ll_IMPACT))) / (1 - np.exp((2 / nobs) * ll_null))
        R2_nagelkerke_COMBINE = (1 - np.exp((2 / nobs) * (ll_null - ll_COMBINE))) / (1 - np.exp((2 / nobs) * ll_null))
        # Bootstrap to calculate delta R2 and 95% CI
        n_boot = 1000
        r2_differences = []
        # Bootstrap loop for R2
        for _ in tqdm(range(n_boot)):
            # 1. Draw a bootstrap sample from the full data
            sample = df1.sample(n=len(df1), replace=True)

            # 2. Extract outcome and predictors
            y_boot = sample[f"target_{IMPACT_out}"]
            X1_boot = sm.add_constant(sample[[f"{IMPACT_model}_{IMPACT_out}"]])  # IMPACT model
            X2_boot = sm.add_constant(sample[[f"{IMPACT_model}_{IMPACT_out}", bio]])  # IMPACT + Cluster model

            try:
                # 3. Fit both models
                model1_boot = sm.Logit(y_boot, X1_boot).fit(disp=0)
                model2_boot = sm.Logit(y_boot, X2_boot).fit(disp=0)

                # 4. Fit null (intercept-only) model
                ll_null = sm.Logit(y_boot, np.ones((len(y_boot), 1))).fit(disp=0).llf
                n = len(y_boot)

                # 5. Compute Nagelkerke R²
                def nagelkerke_r2(ll_model):
                    return (1 - np.exp((2 / n) * (ll_null - ll_model))) / (1 - np.exp((2 / n) * ll_null))

                r2_impact   = nagelkerke_r2(model1_boot.llf)
                r2_combined = nagelkerke_r2(model2_boot.llf)

                # 6. Store the difference
                r2_differences.append(r2_combined - r2_impact)

            except:
                continue  # Skip iteration if model fitting fails
            
        # Analyze the results
        r2_differences = np.array(r2_differences)
        ci_lower = np.percentile(r2_differences, 2.5)
        ci_upper = np.percentile(r2_differences, 97.5)
        mean_diff = np.mean(r2_differences)
        # End Bootstrap for R2
        
        # Bootstrap for delta AIC
        aic_differences = []
        for _ in range(n_boot):
            sample = df1.sample(n=len(df1), replace=True)
            y_boot = sample[f"target_{IMPACT_out}"]
            X1_boot = sm.add_constant(sample[[f"{IMPACT_model}_{IMPACT_out}"]])
            X2_boot = sm.add_constant(sample[[f"{IMPACT_model}_{IMPACT_out}", bio]])
            try:
                model1_boot = sm.Logit(y_boot, X1_boot).fit(disp=0)
                model2_boot = sm.Logit(y_boot, X2_boot).fit(disp=0)
                delta_aic = model1_boot.aic - model2_boot.aic  # positive if model2 is better
                aic_differences.append(delta_aic)
            except:
                continue
            
        # CI for AIC difference
        aic_differences = np.array(aic_differences)
        aic_mean_diff = np.mean(aic_differences)
        aic_ci_low, aic_ci_high = np.percentile(aic_differences, [2.5, 97.5])
        #End bootstrap for delta AIC    
            
        degF_IMPACT  = model1.df_model
        degF_COMBINE = model2.df_model
        degF_Cluster = model3.df_model
        LR_stat = 2*(ll_COMBINE - ll_IMPACT)
        degF_diff = degF_COMBINE - degF_IMPACT
        LR_p  = stats.chi2.sf(LR_stat, degF_diff)
        
        
        AIC_LRT_table.append({
            'Model' : f"{IMPACT_model}_{IMPACT_out}",
            'Biomarker' : bio,
            # 'Cluster AIC/ Delta' : f"{model3.aic:.2f}/ {(model3.aic - model2.aic):.2f}",
            # 'AIC IMPACT/ COMBINED'  : f"{model1.aic:.2f}/ {model2.aic:.2f}",
            'Delta AIC/ (95% CI)' : f"{aic_mean_diff:.2f}/ ({aic_ci_low:.2f}, {aic_ci_high:.2f})",
            # 'IMPACT + Cluster AIC' : f"{model2.aic:.2f}",
            'LRT p_value' : f"{LR_p:.4f}",
            'Nagelkerke R2 IMPACT' : f"{R2_nagelkerke_IMPACT:.2f}",
            # 'Nagelkerke R2 IMPACT/ COMBINED' : f"{R2_nagelkerke_IMPACT:.2f}/ {R2_nagelkerke_COMBINE:.2f}",
            'Delta R2/ (95% CI)' : f"{mean_diff:.2f}/ ({ci_lower:.2f}, {ci_upper:.2f})",
            # 'Nagelkerke R2 COMBINE' : f"{R2_nagelkerke_COMBINE:.2f}",
            # 'Nagelkerke R2 diff' : f"{(R2_nagelkerke_COMBINE - R2_nagelkerke_IMPACT):.2f}",
        })
        
        i = 1
        plt.figure(figsize=(15,11))

        for bio in biomarkers:
            y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"] = df[f"target_{IMPACT_out}"]
            X1 = sm.add_constant(df[f"{IMPACT_model}_{IMPACT_out}"])
            model1 = sm.Logit(y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"], X1).fit()
            # X2 = sm.add_constant(df[[ f"{bio}_1", f"{bio}_2", f"{bio}_3", f"{bio}_4", f"{bio}_5"]])
            X2 = sm.add_constant(df[[f"{IMPACT_model}_{IMPACT_out}", f"{bio}"]])
            model2 = sm.Logit(y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"], X2).fit()
            # X3 = sm.add_constant(df[[f"{IMPACT_model}_{IMPACT_out}", f"{bio}_1", f"{bio}_2", f"{bio}_3", f"{bio}_4", f"{bio}_5"]])
            # model3 = sm.Logit(y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"], X3).fit()
            X3 = sm.add_constant(df[f"{bio}"])
            model3 = sm.Logit(y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"], X3).fit()
            y_pred[f"{IMPACT_model}_{IMPACT_out}_{bio}_IMPACT"] = model1.predict()
            y_pred[f"{IMPACT_model}_{IMPACT_out}_{bio}_COMBINE"] = model2.predict()
            y_pred[f"{IMPACT_model}_{IMPACT_out}_{bio}_Cluster"] = model3.predict()         #df[f"{bio}_{IMPACT_out}"]

            
            #Plot ROC
            # Get true labels and predictions
            y_true_vec = y_true[f"{IMPACT_model}_{IMPACT_out}_{bio}"]
            y_pred1 = y_pred[f"{IMPACT_model}_{IMPACT_out}_{bio}_IMPACT"]
            y_pred2 = y_pred[f"{IMPACT_model}_{IMPACT_out}_{bio}_COMBINE"]
            y_pred3 = y_pred[f"{IMPACT_model}_{IMPACT_out}_{bio}_Cluster"]

            # Compute ROC curves
            fpr1, tpr1, _ = roc_curve(y_true_vec, y_pred1)
            fpr2, tpr2, _ = roc_curve(y_true_vec, y_pred2)
            fpr3, tpr3, _ = roc_curve(y_true_vec, y_pred3)

            # Compute AUCs
            auc1 = roc_auc_score(y_true_vec, y_pred1)
            auc2 = roc_auc_score(y_true_vec, y_pred2)
            auc3 = roc_auc_score(y_true_vec, y_pred3)
            
            ll_IMPACT  = model1.llf
            ll_COMBINE = model2.llf
            ll_Cluster = model3.llf
            
            ll_null = model1.llnull
            print(f"ll_null {ll_null}")
            nobs = model1.nobs
            R2_nagelkerke_IMPACT  = (1 - np.exp((2 / nobs) * (ll_null - ll_IMPACT ))) / (1 - np.exp((2 / nobs) * ll_null))
            R2_nagelkerke_COMBINE = (1 - np.exp((2 / nobs) * (ll_null - ll_COMBINE))) / (1 - np.exp((2 / nobs) * ll_null))
            # Bootstrap to calculate delta R2 and 95% CI
            n_boot = 1000
            r2_differences = []
            # Bootstrap loop for R2          
            for _ in tqdm(range(n_boot)):
                # 1. Draw a bootstrap sample from the full data
                sample = df.sample(n=len(df), replace=True)

                # 2. Extract outcome and predictors
                y_boot = sample[f"target_{IMPACT_out}"]
                X1_boot = sm.add_constant(sample[[f"{IMPACT_model}_{IMPACT_out}"]])  # IMPACT model
                X2_boot = sm.add_constant(sample[[f"{IMPACT_model}_{IMPACT_out}", bio]])  # IMPACT + Cluster model

                try:
                    # 3. Fit both models
                    model1_boot = sm.Logit(y_boot, X1_boot).fit(disp=0)
                    model2_boot = sm.Logit(y_boot, X2_boot).fit(disp=0)

                    # 4. Fit null (intercept-only) model
                    ll_null = sm.Logit(y_boot, np.ones((len(y_boot), 1))).fit(disp=0).llf
                    n = len(y_boot)

                    # 5. Compute Nagelkerke R²
                    def nagelkerke_r2(ll_model):
                        return (1 - np.exp((2 / n) * (ll_null - ll_model))) / (1 - np.exp((2 / n) * ll_null))

                    r2_impact   = nagelkerke_r2(model1_boot.llf)
                    r2_combined = nagelkerke_r2(model2_boot.llf)

                    # 6. Store the difference
                    r2_differences.append(r2_combined - r2_impact)

                except:
                    continue  # Skip iteration if model fitting fails
                
            # Analyze the results
            r2_differences = np.array(r2_differences)
            ci_lower = np.percentile(r2_differences, 2.5)
            ci_upper = np.percentile(r2_differences, 97.5)
            mean_diff = np.mean(r2_differences)
            # End Bootstrap R2
            
            # Bootstrap for delta AIC
            aic_differences = []
            for _ in range(n_boot):
                sample = df.sample(n=len(df), replace=True)
                y_boot = sample[f"target_{IMPACT_out}"]
                X1_boot = sm.add_constant(sample[[f"{IMPACT_model}_{IMPACT_out}"]])
                X2_boot = sm.add_constant(sample[[f"{IMPACT_model}_{IMPACT_out}", bio]])

                try:
                    model1_boot = sm.Logit(y_boot, X1_boot).fit(disp=0)
                    model2_boot = sm.Logit(y_boot, X2_boot).fit(disp=0)

                    delta_aic = model1_boot.aic - model2_boot.aic  # positive if model2 is better
                    aic_differences.append(delta_aic)

                except:
                    continue
                
            # CI for AIC difference
            aic_differences = np.array(aic_differences)
            aic_mean_diff = np.mean(aic_differences)
            aic_ci_low, aic_ci_high = np.percentile(aic_differences, [2.5, 97.5])
            #End bootstrap for delta AIC
            
            degF_IMPACT  = model1.df_model
            degF_COMBINE = model2.df_model
            degF_Cluster = model3.df_model

            LR_stat = 2*(ll_COMBINE - ll_IMPACT)
            degF_diff = degF_COMBINE - degF_IMPACT
            LR_p  = stats.chi2.sf(LR_stat, degF_diff)
            AIC_LRT_table.append({
                'Model' : f"{IMPACT_model}_{IMPACT_out}",
                'Biomarker' : bio,
                # 'Cluster AIC/ Delta' : f"{model3.aic:.2f}/ {(model3.aic - model2.aic):.2f}",
                # 'AIC IMPACT/ COMBINED'  : f"{model1.aic:.2f}/ {model2.aic:.2f}",
                'Delta AIC/ (95% CI)' : f"{aic_mean_diff:.2f}/ ({aic_ci_low:.2f}, {aic_ci_high:.2f})",
                # 'IMPACT AIC/ Delta'  : f"{model1.aic:.2f}/ {(model1.aic - model2.aic):.2f}",
                # 'IMPACT + Cluster AIC' : f"{model2.aic:.2f}",
                'LRT p_value' : f"{LR_p:.4f}",
                'Nagelkerke R2 IMPACT' : f"{R2_nagelkerke_IMPACT:.2f}",
                # 'Nagelkerke R2 IMPACT/ COMBINED' : f"{R2_nagelkerke_IMPACT:.2f}/ {R2_nagelkerke_COMBINE:.2f}",
                'Delta R2/ (95% CI)' : f"{mean_diff:.2f}/ ({ci_lower:.2f}, {ci_upper:.2f})",
                # 'Nagelkerke R2 IMPACT' : f"{R2_nagelkerke_IMPACT:.2f}",
                # 'Nagelkerke R2 COMBINE' : f"{R2_nagelkerke_COMBINE:.2f}",
                # 'Nagelkerke R2 diff' : f"{(R2_nagelkerke_COMBINE - R2_nagelkerke_IMPACT):.2f}",

            })
            
            # Plot all ROC curves
            # plt.figure(figsize=(4,3))
            # plt.subplot(2,3,i)
            # i += 1
            # plt.plot(fpr1, tpr1, label=f"IMPACT (AUC={auc1:.3f})", color='blue')
            # plt.plot(fpr2, tpr2, label=f"{bio}   (AUC={auc2:.3f})", color='red')
            # plt.plot(fpr3, tpr3, label=f"COMBINED (AUC={auc3:.3f})", color='green')
            # plt.plot([0,1], [0,1], linestyle='--', color='gray')

            # plt.xlabel("False Positive Rate")
            # plt.ylabel("True Positive Rate")
            # plt.title(f"{IMPACT_model} {IMPACT_out} {bio}")
            # # plt.title(f"ROC Curve - {IMPACT_model} {IMPACT_out} {bio}")
            # plt.legend(loc='lower right')
            # plt.grid(True)

            # Save plot
            # filename = f"ROC_{IMPACT_model}_{IMPACT_out}_{bio}.png"
            # plt.savefig(filename)
        # plt.show()

# df_y_true = pd.DataFrame(y_true)
# df_y_pred = pd.DataFrame(y_pred)
df_AIC_LR = pd.DataFrame(AIC_LRT_table)







In [ ]:
# df_y_true.to_csv('../../IMPACT/y_true_prob.csv', header=True, index=False)
# df_y_pred.to_csv('../../IMPACT/y_pred_prob.csv', header=True, index=False)
# df_y_pred.shape
df1.shape
# df_AIC_LR.to_csv('../../IMPACT/AIC_LRT_R2_bio_simplified.csv', header=True, index=False)
df_AIC_LR


,Model,Biomarker,Delta AIC/ (95% CI),LRT p_value,Nagelkerke R2 IMPACT,Delta R2/ (95% CI)
0,core_unfavor,composite,"16.87/ (2.33, 36.69)",0.0000,0.34,"0.10/ (0.03, 0.21)"
1,core_unfavor,GFAP,"16.75/ (3.62, 36.68)",0.0000,0.33,"0.06/ (0.02, 0.12)"
2,core_unfavor,NFL,"16.88/ (3.45, 36.82)",0.0000,0.33,"0.06/ (0.02, 0.12)"
3,core_unfavor,Tau,"16.82/ (2.69, 35.16)",0.0000,0.33,"0.06/ (0.02, 0.11)"
4,core_unfavor,UCH-L1,"9.46/ (-0.68, 25.90)",0.0014,0.33,"0.04/ (0.00, 0.09)"
5,core_unfavor,S100B,"9.78/ (-0.19, 27.10)",0.0010,0.33,"0.04/ (0.00, 0.08)"
6,core_unfavor,NSE,"1.27/ (-2.00, 9.86)",0.1319,0.33,"0.01/ (0.00, 0.04)"
7,core_mortal,composite,"21.43/ (6.14, 41.23)",0.0000,0.24,"0.16/ (0.06, 0.29)"
8,core_mortal,GFAP,"11.06/ (0.81, 27.07)",0.0005,0.23,"0.06/ (0.01, 0.13)"
9,core_mortal,NFL,"15.90/ (2.43, 36.81)",0.0000,0.23,"0.08/ (0.02, 0.15)"
